In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", ]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)

site = Website("website_url")

prompt = "You are an assistant that analyzes the contents of a website and based on the website I want you to provide a cover letter for an internship candidate who is interested in data science, ignoring text that might be navigation related. Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; provide a cover letter for an internship candidate who is interested in data science in markdown. Add news and announcements too.\n\n"
    user_prompt += website.text
    return user_prompt

print(user_prompt_for(site)) # Displaying the page

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# messages_for(site)
# Delete the '#' for debugging


def coverLetter(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

# coverLetter("url")
# Delete the '#' for debugging


def display_letter(url):
    summary = coverLetter(url)
    display(Markdown(summary))

display_letter("same_url_as_before")